In [1]:
import cnvpytor
import subprocess
print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del_400_1_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1129759

In [3]:
read_count = line_count * 50 *30/202
read_count = round(read_count)
read_count

8389300

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700236734
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 56487872


b'chr22\t1240\tG\tT\t-\nchr22\t2148\tC\tY\t+\nchr22\t3357\tC\tY\t+\nchr22\t3473\tC\tS\t+\nchr22\t4916\tC\tT\t-\nchr22\t5450\tA\tM\t+\nchr22\t5498\tA\tW\t+\nchr22\t5590\t-\tACG\t-\nchr22\t7938\t-\tC\t+\nchr22\t9708\tC\tM\t+\nchr22\t11194\tC\t-\t-\nchr22\t12339\tC\tA\t-\nchr22\t12980\tC\tT\t-\nchr22\t14703\tA\tM\t+\nchr22\t14926\tT\tY\t+\nchr22\t15035\tA\tR\t+\nchr22\t18054\tG\tS\t+\nchr22\t18224\t-\tGG\t+\nchr22\t19753\t-\tG\t-\nchr22\t20395\tA\tW\t+\nchr22\t21839\tA\tC\t-\nchr22\t21917\tC\t-\t-\nchr22\t21927\t-\tGGGT\t-\nchr22\t22631\tT\tY\t+\nchr22\t23527\tA\tC\t-\nchr22\t24230\t-\tC\t-\nchr22\t24269\tA\tM\t+\nchr22\t24285\tC\tS\t+\nchr22\t24705\tC\tY\t+\nchr22\t25580\tC\tS\t+\nchr22\t26201\tG\tA\t-\nchr22\t26388\tT\tC\t-\nchr22\t28245\tA\tR\t+\nchr22\t30443\tG\tS\t+\nchr22\t30631\tA\tR\t+\nchr22\t34463\tA\tW\t+\nchr22\t34774\tG\tR\t+\nchr22\t35411\t-\tG\t+\nchr22\t35434\tC\tY\t+\nchr22\t36941\tT\tY\t+\nchr22\t37483\tA\tM\t+\nchr22\t37954\tA\tR\t+\nchr22\t38289\tT\tY\t+\nchr22\t40432\

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 10 files and 1 in-memory blocks...


In [27]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [ ]:
app.rd(["biasOut_sorted.bam"])

In [8]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [9]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	2.0128	1.485153e-06	8.284846e-67	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	2.9752	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13480601-13512800	32200	8.0759	0.000000e+00	2.123859e-76	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:18662501-18667700	5200	1.9693	0.000000e+00	1.700012e-10	1.495801e-05	4.223529e-03	0.0005	0.0000	1814800
duplication	chr22:18669601-18672600	3000	1.7962	3.104640e-03	7.542452e+01	1.379280e+03	1.807605e-02	0.0009	0.0000	1821900
duplication	chr22:18680401-18682700	2300	2.0378	4.936669e-03	2.953449e-03	1.266231e+05	1.437308e-07	0.0000	0.0000	1826900
duplication	chr22:18704701-18881400	176700	5.3326	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	1628200
duplication	chr22:20327001-20328500	1500	2.2304	4.538190e-02	5.047995e-33	1.000000e+00	1.000000e+00	0.0000	0.0000	181100
duplication	chr22:20331101-20334900	3800	1.567

In [11]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [19]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 100")
for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 100
duplication,chr22,643201,644800,1600,2.0128391238926295,0.0
duplication,chr22,8039401,8047900,8500,2.9752225975788567,0.0
duplication,chr22,13480601,13512800,32200,8.075906086240241,0.0
duplication,chr22,18662501,18667700,5200,1.9692721088438516,0.000474158368895211
duplication,chr22,18669601,18672600,3000,1.796153217875886,0.0009010361916203634
duplication,chr22,18680401,18682700,2300,2.0377829961003244,0.0
duplication,chr22,18704701,18881400,176700,5.332639010002464,0.00013574711172245044
duplication,chr22,20327001,20328500,1500,2.230379766141817,0.0
duplication,chr22,20331101,20334900,3800,1.567406622358018,0.0013583265417006249
duplication,chr22,20336601,20340900,4300,1.790760564927301,0.0012607690691321706
duplication,chr22,20341301,20344500,3200,2.2270852147181106,0.0009085746734809767
duplication,chr22,20345801,20347800,2000,1.821782846691057,0.0
duplication,chr22,203

In [20]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 1000")
for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 1000
duplication,chr22,8040001,8048000,8000,2.9657218552288995,0.0
duplication,chr22,13481001,13513000,32000,8.068034946741731,0.0
duplication,chr22,18658001,18683000,25000,1.6097772608681964,0.0005186204695509944
duplication,chr22,18705001,18881000,176000,5.310545972062351,0.00010512918394763532
duplication,chr22,20326001,20368000,42000,1.6602521364795269,0.000506323287419017
duplication,chr22,20609001,20655000,46000,2.0714712630225605,0.00010121798981072236
duplication,chr22,21466001,21505000,39000,1.742857974862873,0.0001890113524943592
duplication,chr22,21522001,21559000,37000,1.6725015106455614,0.0018434376217058288
duplication,chr22,21561001,21686000,125000,1.9202055004564107,0.0005421977080432687
duplication,chr22,23608001,23613000,5000,3.9708016441181617,0.0
duplication,chr22,28466001,28599000,133000,5.963132620186792,5.067909993918508e-05
duplication,chr22,30804001,3098

In [21]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 10,000")
for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 10,000
duplication,chr22,18650001,18880000,230000,4.275761561957504,0.0001531182532269672
duplication,chr22,21460001,21680000,220000,1.7654950956471787,0.00048545586568505845
duplication,chr22,28460001,28600000,140000,5.692996308105824,5.0146880212141365e-05
duplication,chr22,30800001,30980000,180000,5.893362490755551,1.5188704464264017e-05
duplication,chr22,35940001,36100000,160000,4.00046769669379,1.756128889825491e-05
duplication,chr22,36110001,36200000,90000,3.961175076928347,4.476586334324897e-05
duplication,chr22,44170001,44460000,290000,4.002050110899782,2.214989665688841e-05
duplication,chr22,46380001,46670000,290000,3.001015004284367,1.664866144761961e-05


In [22]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 100,000")
for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 100,000


In [23]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 1,000,000")
for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 1,000,000


In [24]:
print('type,chromosome_name,start,end,size,cnv_level,q0')
print(f"{filename=}, Histogram Size: 10,000,000")
for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=',')

type,chromosome_name,start,end,size,cnv_level,q0
filename='test_with_del_400_1_formated.fa', Histogram Size: 10,000,000


In [28]:
del app